## Introduction

Milan, as well as being a fashionable city, is known for being full of bars and cafés, especially in the city centre. This project aims to estimate the best localization to open such a business in Milan city, just nearby Duomo/Scala Area.

## Problem

As the goal of this notebook is to find the best location for a new bar/caffé in Milan city.
In order to do so we are going to search for perfect spot in the city, close to Duomo Area but however with few or no venues in the vicinity 

## Data 

In this assignment we will use Foursquare to find bars and cafés in the city of Milan. Further data that could be used are those related to activities of interest or universities.
In addition, we could use location data of the various metro stations, so as to find an area in the centre, near a metro where there are not yet many bars.

### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import math
# !pip install pyproj
import pyproj

#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Foursquare Credentials

In [2]:
CLIENT_ID = 'LTRMM3GFOLP24MVB5L4UC4G3W4UI3TMZTNRPLNGVWOXSGKAA' # your Foursquare ID
CLIENT_SECRET = 'JZJU22GVRXADVBCDKMNN2LJ5UU3VIGKQEHLGPPSJFTU1SFNZ' # your Foursquare Secret
ACCESS_TOKEN = 'BGOSGXQB1KTNOCNDEYCM0WYHHRYWQXNT3PY0QLMJEQXFW1MR' # your FourSquare Access Token
VERSION = '20180605'
LIMIT = 127
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LTRMM3GFOLP24MVB5L4UC4G3W4UI3TMZTNRPLNGVWOXSGKAA
CLIENT_SECRET:JZJU22GVRXADVBCDKMNN2LJ5UU3VIGKQEHLGPPSJFTU1SFNZ


## City partitionning

In [11]:
address = 'Milan, Italy'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
Milan_center = [latitude, longitude]
Milan_center
Milan_duomo = [45.27,9.11]

In [8]:

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Milan center longitude={}, latitude={}'.format(Milan_center[1], Milan_center[0]))
x, y = lonlat_to_xy(Milan_center[1], Milan_center[0])
print('Milan center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Milan center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Milan center longitude=9.1905, latitude=45.4668
Milan center UTM X=45868.32335936738, Y=5051248.231440709
Milan center longitude=9.1905, latitude=45.46679999999999


<ipython-input-8-5be3aef5dbf9>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-8-5be3aef5dbf9>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)


In [21]:
Milan_center_x, Milan_center_y = lonlat_to_xy(Milan_center[1], Milan_center[0]) # City center in Cartesian coordinates
nb_k = 10
radius = 100
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = Milan_center_x - radius*10
x_step = radius*2
y_min = Milan_center_y - radius*2 - (int(nb_k/k)*k*radius*2 - radius*10)/2
y_step = radius*2 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []

for i in range(0, int(nb_k/k)):
    y = y_min + i * y_step
    x_offset = radius if i%2==0 else 0
    for j in range(0, nb_k):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Milan_center_x, Milan_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)
            
map_milan = folium.Map(location=Milan_center, zoom_start=15)
folium.Marker(Milan_center, popup='City center').add_to(map_milan)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=radius, color='blue', fill=False).add_to(map_milan)
map_milan

<ipython-input-8-5be3aef5dbf9>:4: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
<ipython-input-8-5be3aef5dbf9>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-8-5be3aef5dbf9>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-8-5be3aef5dbf9>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
<ipython-input-8-5be3aef5dbf9>:10: DeprecationWarning: This funct

## Coffees visualization

In [14]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Milan",
    "radius": 1000,
    "limit": 127}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [15]:
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [16]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Milan', 'Milan')

In [17]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '57d52948498ebceb900cf11c',
  'name': 'Pasticceria Marchesi',
  'location': {'address': 'Galleria Vittorio Emanuele II',
   'lat': 45.46561245831536,
   'lng': 9.190091221105353,
   'labeledLatLngs': [{'label': 'display',
     'lat': 45.46561245831536,
     'lng': 9.190091221105353}],
   'postalCode': '20121',
   'cc': 'IT',
   'neighborhood': 'Duomo',
   'city': 'Milano',
   'state': 'Lombardia',
   'country': 'Italia',
   'formattedAddress': ['Galleria Vittorio Emanuele II',
    '20121 Milano Lombardia',
    'Italia']},
  'categories': [{'id': '5744ccdfe4b0c0459246b4e2',
    'name': 'Pastry Shop',
    'pluralName': 'Pastry Shops',
    'shortName': 'Pastry',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 're

In [18]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': 'Galleria Vittorio Emanuele II', 'lat': 45.46561245831536, 'lng': 9.190091221105353, 'labeledLatLngs': [{'label': 'display', 'lat': 45.46561245831536, 'lng': 9.190091221105353}], 'postalCode': '20121', 'cc': 'IT', 'neighborhood': 'Duomo', 'city': 'Milano', 'state': 'Lombardia', 'country': 'Italia', 'formattedAddress': ['Galleria Vittorio Emanuele II', '20121 Milano Lombardia', 'Italia']}
{'address': 'Piazza Cordusio 3', 'crossStreet': 'Via Orefici', 'lat': 45.46492026479228, 'lng': 9.186152815818787, 'labeledLatLngs': [{'label': 'display', 'lat': 45.46492026479228, 'lng': 9.186152815818787}], 'postalCode': '20123', 'cc': 'IT', 'city': 'Milano', 'state': 'Lombardia', 'country': 'Italia', 'formattedAddress': ['Piazza Cordusio 3 (Via Orefici)', '20123 Milano Lombardia', 'Italia']}
{'address': 'Via Gaetano Giardino 1', 'lat': 45.462816426798085, 'lng': 9.189022685823147, 'labeledLatLngs': [{'label': 'display', 'lat': 45.462816426798085, 'lng': 9.189022685823147}], 'postalCode':

,uid,name,shortname,address,postalcode,lat,lng
0,57d52948498ebceb900cf11c,Pasticceria Marchesi,Pastry,,20121,45.465612,9.190091
1,58b5d3364e31c52f888a0bed,Starbucks Reserve Roastery,Coffee Shop,,20123,45.464920,9.186153
2,576828cc498ec1d7fb395d70,Caffè Napoli,Café,,20123,45.462816,9.189023
3,59c2232bd48ec17dd4178369,Lavazza Coffee Design,Coffee Shop,,20122,45.466274,9.190975
4,4be181c240d676b02de504ee,Princi,Bakery,,20123,45.463029,9.187968


In [19]:
from folium import plugins
map_milan = folium.Map(location=[45.4668, 9.1905], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_milan)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_milan.add_child(plugins.HeatMap(hm_data))

map_milan

In [22]:
map_milan = folium.Map(location=[45.4668, 9.1905], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_milan)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_milan.add_child(plugins.HeatMap(hm_data))

folium.Marker(Milan_center, popup='City center').add_to(map_milan)
for lat, lon in zip(latitudes, longitudes):
    folium.Circle([lat, lon], radius=radius, color='blue', fill=False).add_to(map_milan)


map_milan

<a id="item1"></a>
